# Tutorial 6: Inspecting predicted proteome allocation

In this tutorial we will calculate the proteome allocation predicted by the _Bacillus subtilis_ ME-model.

## Import libraries

In [ ]:
from coralme.builder.main import MEBuilder
from coralme.util.flux_analysis import flux_based_reactions
import coralme
import pandas
import tqdm

## Load

Load the ME-model coming out of the Troubleshooter

In [ ]:
me = coralme.io.json.load_json_me_model("MEModel-step3-bsubtilis-TS.json")

## Solve

In [ ]:
me.reactions.EX_glc__D_e.bounds = (-10,0)

In [ ]:
me.optimize()

## Proteome allocation

We here use some of the functions provided in coralME to calculate proteome allocation. This part of the analysis can take several minutes to run. The algorithm needs to traverse the ME-network finding where all proteins go (accounting for final destinations only) and calculate the partitioning from the fluxes.

### Calculate protein to complex partitioning

Here we calculate the final complexes to which 1 unit of every protein goes

In [ ]:
from coralme.builder.helper_functions import get_partitioning,get_immediate_partitioning

In [ ]:
ComplexPartitioning = {}
for protein in tqdm.tqdm(me.metabolites.query("^protein_")):
    if not isinstance(protein,coralme.core.component.TranslatedGene):
        continue
    partitioning = get_partitioning(protein,seen = set(),final_fraction=1.0)
    if not partitioning:
        continue
    ComplexPartitioning[protein.id] = {k.id:v for k,v in partitioning}

In [ ]:
pandas.DataFrame.from_dict(ComplexPartitioning).fillna(0.).head()

### Calculate protein to function partitioning

Here we calculate the final reactions to which 1 unit of every protein goes

In [ ]:
ReactionPartitioning = {}
for protein,partitioning in tqdm.tqdm(ComplexPartitioning.items()):
    ReactionPartitioning[protein] = {}
    tmp = pandas.Series()
    for k,fraction in partitioning.items():
        ip = {k.id:v for k,v in get_immediate_partitioning(me.get(k)).items()}
        tmp = tmp.append(pandas.Series(ip) * fraction)
        if tmp.duplicated().any():
            tmp = tmp.to_frame().reset_index().groupby("index").sum()[0]
    ReactionPartitioning[protein] = tmp.to_dict()

In [ ]:
ReactionPartitioning = pandas.DataFrame.from_dict(ReactionPartitioning).fillna(0.)
ReactionPartitioning.head()

### Get protein translation fluxes

In [ ]:
dct = {}
for r in me.query("translation_"):
    p = "protein_" + r.id.split("translation_")[1]
    dct[p] = r.flux * me.get(p).formula_weight/1000
TranslationFluxes = pandas.Series(dct)
TranslationFluxes.head()

### Combine results into one dataframe

Now we calculate the contributions of all proteins to every reaction flux

In [ ]:
FluxPartitioning = (ReactionPartitioning*TranslationFluxes).dropna(axis=1)
FluxPartitioning.head()

The sum of all fluxes should add up to the protein biomass

In [ ]:
FluxPartitioning.sum().sum()

In [ ]:
me.solution.fluxes["protein_biomass_to_biomass"]

### Get reaction to class dictionary

Here we classify all reactions into categories to simplify the analysis

In [ ]:
def get_function(r,metabolics=False):
    if isinstance(r,coralme.core.reaction.MetabolicReaction) and hasattr(r,'subsystem'):
        if r.subsystem and metabolics:
            return 'Metabolic:' + r.subsystem
        return 'Metabolic'
    if isinstance(r,coralme.core.reaction.TranslationReaction):
        return 'Translation'
    elif isinstance(r,coralme.core.reaction.TranscriptionReaction):
        return 'Transcription'
    elif isinstance(r,coralme.core.reaction.tRNAChargingReaction):
        return 'tRNA-Charging'
    elif isinstance(r,coralme.core.reaction.PostTranslationReaction):
        return 'Post-translation'
    elif isinstance(r,coralme.core.reaction.SummaryVariable):
        return 'Biomass'
    return "No_function"

In [ ]:
ReactionToClass = pandas.DataFrame(index=[r.id for r in me.reactions]).reset_index()
ReactionToClass["Function1"] = ReactionToClass["index"].apply(lambda x:get_function(me.get(x)))
ReactionToClass["Function2"] = ReactionToClass["index"].apply(lambda x:get_function(me.get(x),metabolics=True))
ReactionToClass = ReactionToClass.set_index("index").sort_index()
ReactionToClass.head()

### Visualize proteome allocation

Now we annotate it for further analysis

In [ ]:
AnnotatedFluxPartitioning = pandas.concat([FluxPartitioning,ReactionToClass],axis=1).dropna(axis=0)
AnnotatedFluxPartitioning.head()

In [ ]:
AnnotatedFluxPartitioning.groupby("Function1").sum().sum(axis=1).sort_values().plot.pie()

In [ ]:
AnnotatedFluxPartitioning.groupby("Function2").sum().sum(axis=1).sort_values().plot.pie()

In [ ]:
FluxPartitioning.sum().sort_values()

In [ ]:
AnnotatedFluxPartitioning[['protein_BSU22890',"Function2"]].sort_values("protein_BSU22890")